## Importing libraries

In [1]:
import pandas as pd
from docx2pdf import convert
import pdfminer
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pdfplumber
from docx import *
import pandas as pd
import re
from spacy_annotator.pandas_annotations import annotate as pd_annotate
import glob
import os

## Setting paths for input and output

In [5]:
convert("Training_data/", output_path="PDFs/")

In [7]:
default_directory = os.getcwd()

In [8]:
os.chdir("Training_data/")

## Iterating through the directory

In [150]:
mylist = [f for f in glob.glob("*.docx")]

In [151]:
mylist

['343492954-Rent-Agreement-3E.pdf.docx',
 '54945838-Rental-Agreement.pdf.docx',
 '288024755-Rental-Agreement-1.pdf.docx',
 '267005869-Rental-Tenant.pdf.docx',
 '62126501-Rental-Agreement.pdf.docx',
 '50070534-RENTAL-AGREEMENT (1).pdf.docx',
 '251798216-Rental-Agreement-Format1.pdf.docx',
 '116950326-December-2012-Rental-Agreement.pdf.docx',
 '54770958-Rental-Agreement.pdf.docx',
 '203615996-Rental-Agreement-Format.pdf.docx',
 '248636461-Rental-Agreement.pdf.docx',
 '195231682-This-RENTAL-AGREEMENT-is-Made-and-Executed-on-24th-Day-of-September.pdf.docx',
 '269760124-97646-41223-Rental-Agreement.pdf.docx',
 '249104436-House-Rental-Agreement.pdf.docx',
 '36199312-Rental-Agreement.pdf.docx',
 '99699504-Rental-Agreement-English-Model.pdf.docx',
 '323828497-Rental-Agreement-Micky.pdf.docx',
 '170499354-Anand-Nagar-Agreement.pdf.docx',
 '6683127-House-Rental-Contract-GERALDINE-GALINATO-v2-Page-1.pdf.docx',
 '56736420-Rental-Agreement.pdf.docx',
 '63793679-Rental-Agreement.pdf.docx',
 '7711235

## Test function to extract data from a file. This can later be scaled up to work on each file in directory.

In [12]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [100]:
getText(mylist[11]).split("rn")

['\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRENTAL AGREEMENT\nTHIS DEED OF RENTAL AGREEMENT executed at Chennai this 19th day of August 2013 between:\nMrs. M. Geetha, W/o. V. Manoharan, aged about 53 years, residing at No.62-D, Saravana Flats, 1st Floor, Pinagapani Street, Devaki Nagar, Kundrathur, Chennai-600 069. Hereinafter called the owner (Which expression shall, wherever the context so requires, be deemed to mean and include her legal heirs, executors, administrators and assignees) of the one part.\nAND\nTo and in favour of M/s. Siruvani Traders Private Limited Represented by Mr. J. L. Saravana, aged about 33 years S/o M S Jambulingam, residing at No. 13/5, Janakiraman Street, Muthurangam Block, Jafferkhanpet, Ashok Nagar, Chennai - 600 083, hereinafter called the tenant (which expression shall, wherever the context so requires be deemed to mean and include his successors- in- interest and assignees) of the other part witnesses as follows:\n-2-\nWhereas 

## The function works but we need the data in a dataframe. So the below line is condensed version of getting the Dataframe from a string.

In [110]:
pd.DataFrame(pd.Series(getText(mylist[11])))

,0
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


## Scaling up

In [156]:
entire_text_df = []
for i in range(len(mylist)):
    entire_text_df.append(pd.Series(getText(mylist[i])))

## Assigning results and appending them.

In [165]:
df = pd.DataFrame(entire_text_df).rename(columns={0:'full_text'})

In [164]:
# df = pd.DataFrame.from_dict({'full_text' : pd.Series(getText(mylist[11]))})

## Since the data files are in PDF format. 
## I have converted and this is the flow, PDF>Extraction to text> Conversion to Dataframe> Coerce to dict and assign to a large appended Dataframe

## Below is the manual annotation for each file (43 PDFs).

### The widget will be visible based on running state. Running live images have been attached to the "file-name" doc.

## The annotations cannot be in any format for a NLP-NER model. They have to be a list of tuples, these tuples are nested in the form (string, entity , start location, end location).

### Tried multiple different approaches.
### ** Look at the summary after the block.

In [166]:
# Annotations
pd_dd = pd_annotate(df,
            col_text = 'full_text',     # Column in pandas dataframe containing text to be labelled
            labels = ['AggValue', 'AggStart','AggEnd','PartyOne','PartyTwo','Notice'], # List of labels
            sample_size=1,              # Size of the sample to be labelled
            delimiter='~',              # Delimiter to separate entities in GUI
            model = None,               # spaCy model for noisy pre-labelling
            regex_flags=re.IGNORECASE   # One (or more) regex flags to be applied when searching for entities in text
            )

Instructions 
Input must be in the following format: 
labelA: item1, item2; 
labelB: itemX, itemZ; 

If no entities in text, leave as is and press submit. 
Similarly, if no entities for a particular label, leave as is (or delete the line for that label). 

Buttons: 
*submit inserts new annotation (or overwrites existing one if one is present). 
*skip moves forward and leaves empty string (or existing annotation if one is present). 
*finish terminates the annotation session.


HTML(value='-1 examples annotated, 44 examples left')

Textarea(value='AggValue: insert;\nAggStart: insert;\nAggEnd: insert;\nPartyOne: insert;\nPartyTwo: insert;\nN…

Output()

## The annotations have been appended into a series which is joined to the previous dataframe.

In [173]:
annotseries = pd_dd['annotations']
annotdf = pd_dd

In [180]:
annotdf.to_csv("annotations.csv")

##  This is the final df with annotations and has been outputted as "annotations.csv".
## The format of annotations can be found by looking at the individual cell of annotations column.

In [174]:
annotdf

,full_text,annotations
0,STANDARD REAL ESTATE RENTAL AGREEMENT\nThis Ag...,(STANDARD REAL ESTATE RENTAL AGREEMENT\nThis A...
1,HOUSE RENTAL AGREEMENT\nRental Agreement made ...,(HOUSE RENTAL AGREEMENT\nRental Agreement made...
2,"Rental Agreement\nThis agreement made this 19""...",(Rental Agreement\nThis agreement made this 19...
3,\nTENANCY AGREEMENT\n\nThis Tenancy Agreement ...,(\nTENANCY AGREEMENT\n\nThis Tenancy Agreement...
4,RENTAL AGREEMENT\nTHE AGREEMENT\nThe landlord ...,(RENTAL AGREEMENT\nTHE AGREEMENT\nThe landlord...
5,RENTAL AGREEMENT\nThis Rental Agreement made a...,(RENTAL AGREEMENT\nThis Rental Agreement made ...
6,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRENTAL A...,(\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRENTAL ...
7,HOUSE RENTAL AGREEMENT\nThis Rental Agreement ...,(HOUSE RENTAL AGREEMENT\nThis Rental Agreement...
8,RENTAL AGREEMENT\nParties\nThe parties to this...,(RENTAL AGREEMENT\nParties\nThe parties to thi...
9,\n\nROOM RENTAL AGREEMENT\nThis is a legally b...,(\n\nROOM RENTAL AGREEMENT\nThis is a legally ...


## Populating the tuples by nesting them by location to mimick the exact skeletal structure of the NER trainer.

In [181]:
annotuple = []
for i in range(annotdf["annotations"].shape[0]):
    annotuple.append(annotdf["annotations"].iloc[i])

In [184]:
annotuple_final = tuple(annotuple)

## Assigning the tuple as training data.

In [189]:
TRAIN_DATA = annotuple

## The function below trains the NER model with custom manual annotations by creating a NER pipeline and using spacy's inbuilt algo for entity recognition which takes the tuple list of annotations. The fit has a dropout value (0.2) to avoid overfit.

In [190]:
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

## The tuple standard format, perfectly fits into the spacy specification.

In [191]:
annotuple

[('STANDARD REAL ESTATE RENTAL AGREEMENT\nThis Agreement made this 11 day of December, 2008, is between PROSPER 4 LESS,LLC\t (hereinafter referred to as\n"Management") and Jose Luis Salcido owner(hereinafter referred to as "Resident"). Management rents to Resident, and Resident rents from Management, property located at:\nAddressl6714 MOSSDALE BLVD.LANCASTER, State: California, Zip: 93535, the full legal description of which is the same as recorded with the Clerk of the Superior Court of Kern County, and is made a part hereof by reference (hereinafter referred to as the "Property"), under the following terms and conditions:\nTERM: The initial term of this Agreement shall be for $700.00 Dollars per months beginning on the 15 day of DECEMBER, 2008.\nCONDITION: Resident accepts Property in its present "AS-IS" condition and acknowledges that Resident has received a list of any existing damages to Property, been given the right to inspect same, and has approved said list except as previousl

## Training the model with 100 iterations and the losses are printed through the function.

In [192]:
import random
prdnlp = train_spacy(TRAIN_DATA, 100)

Statring iteration 0
{'ner': 7644.150391501519}
Statring iteration 1
{'ner': 1398.0613526066404}
Statring iteration 2
{'ner': 5011.345945935511}
Statring iteration 3
{'ner': 2420.945880664166}
Statring iteration 4
{'ner': 3413.2436084559176}
Statring iteration 5
{'ner': 4829.112955668291}
Statring iteration 6
{'ner': 4044.0342490845346}
Statring iteration 7
{'ner': 3483.026417168183}
Statring iteration 8
{'ner': 3657.345768572367}
Statring iteration 9
{'ner': 3138.3011355750054}
Statring iteration 10
{'ner': 2859.130341049936}
Statring iteration 11
{'ner': 3723.887276487717}
Statring iteration 12
{'ner': 3052.7944301775565}
Statring iteration 13
{'ner': 1566.852116390254}
Statring iteration 14
{'ner': 2777.122884001488}
Statring iteration 15
{'ner': 885.0190954142001}
Statring iteration 16
{'ner': 1339.3809387462009}
Statring iteration 17
{'ner': 1070.7870012368794}
Statring iteration 18
{'ner': 545.929514824466}
Statring iteration 19
{'ner': 481.5410712187019}
Statring iteration 20
{'

In [193]:
prdnlp

### Changing directory to point to validation docs

In [195]:
os.chdir("/Users/sahmehta/Downloads/Zycus/Validation_Data/")

In [196]:
mylist_val = [f for f in glob.glob("*.docx")]

mylist_val

['24158401-Rental-Agreement.pdf.docx',
 '228094620-Rental-Agreement.pdf.docx',
 '195231682-This-RENTAL-AGREEMENT-is-Made-and-Executed-on-24th-Day-of-September.pdf.docx',
 '63793679-Rental-Agreement.pdf.docx',
 '239419594-Rental-Agreement.pdf.docx',
 '95980236-Rental-Agreement.pdf.docx',
 '156155545-Rental-Agreement-Kns-Home.pdf.docx',
 '269135973-Udaya-Rental-Agreement.pdf.docx']

In [238]:
#Test your text
doc = prdnlp(getText(mylist_val[0]))

### Demo entity from model

In [239]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

1st day of April 2008 68 89 AggStart
1-04-08) by and between 91 114 AggEnd
Rs 80,000 1818 1827 AggValue
two months notice 2054 2071 Notice
two months notice 2147 2164 Notice


In [210]:
from spacy.gold import GoldParse
from spacy.scorer import Scorer

In [219]:
def evaluate(ner_model, examples):
    scorer = Scorer()
    for sents, ents in examples:
        doc_gold = ner_model.make_doc(sents)
        gold = GoldParse(doc_gold, entities=ents['entities'])
        pred_value = ner_model(sents)
        scorer.score(pred_value, gold)
    return scorer.scores

In [220]:
results = evaluate(prdnlp, TRAIN_DATA)
print(results)

{'uas': 0.0, 'las': 0.0, 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'ents_p': 94.68085106382979, 'ents_r': 94.68085106382979, 'ents_f': 94.68085106382979, 'ents_per_type': {'AggValue': {'p': 100.0, 'r': 93.33333333333333, 'f': 96.55172413793103}, 'PartyOne': {'p': 100.0, 'r': 91.30434782608695, 'f': 95.45454545454545}, 'PartyTwo': {'p': 95.65217391304348, 'r': 100.0, 'f': 97.77777777777777}, 'AggStart': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'Notice': {'p': 88.88888888888889, 'r': 80.0, 'f': 84.21052631578948}, 'AggEnd': {'p': 57.14285714285714, 'r': 100.0, 'f': 72.72727272727273}}, 'tags_acc': 0.0, 'token_acc': 100.0, 'textcat_score': 0.0, 'textcats_per_cat': {}}


## Evaluation gives a high recall as can be seen below.
## The F-beta scores are 95.45, 97.8, 84.2 and 72.8 respectively.
## The training data has lesser number of files and hence the results although an encouragement, should therefore be tried to generalise based on a higher number of training files. 

### We have a good structure to build upon and hence this can act as a solid foundation for when there are higher number of files available.

In [221]:
results.to

{'uas': 0.0,
 'las': 0.0,
 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'ents_p': 94.68085106382979,
 'ents_r': 94.68085106382979,
 'ents_f': 94.68085106382979,
 'ents_per_type': {'AggValue': {'p': 100.0,
   'r': 93.33333333333333,
   'f': 96.55172413793103},
  'PartyOne': {'p': 100.0, 'r': 91.30434782608695, 'f': 95.45454545454545},
  'PartyTwo': {'p': 95.65217391304348, 'r': 100.0, 'f': 97.77777777777777},
  'AggStart': {'p': 100.0, 'r': 100.0, 'f': 100.0},
  'Notice': {'p': 88.88888888888889, 'r': 80.0, 'f': 84.21052631578948},
  'AggEnd': {'p': 57.14285714285714, 'r': 100.0, 'f': 72.72727272727273}},
 'tags_acc': 0.0,
 'token_acc': 100.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {}}

### Model is saved to Disk. Name can be entered by the user.

In [222]:
# Save our trained Model
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)

Enter your Model Name: NER-Rental


_

### While above approach was focussed on getting the model and annotations right. Below is non-standard (non formatted code) to output annotations into a proper csv.
## Outputs are saved into a folder for ease of access : OutputCSVs.

In [328]:
os.chdir('/Users/sahmehta/Downloads/Zycus/')

In [330]:
import re
def clean(text):
    text=text.replace('/','.')
    text=text.replace('\\','.')
    text=text.replace('. ','.')
    text=text.replace(', ','')
    text=text.replace(',','')
    text=text.replace('-','.')
    text=text.replace('/','.')
    text=text.replace('. ','.')
    #text=text.lower()
    return(text)

In [331]:
# prepair data for training
data = []
train_labels=pd.read_csv('TrainingTestSet.csv')
data_index = list(train_labels.columns)
count = 0
new_data = []
ph_1,ph_2,ph_3,ph_4,ph_5,ph_6 = [],[],[],[],[],[]
for i in os.listdir('Training_data/'):
    doc = docx.Document('Training_data/'+i)  # Creating word reader object.
    paragraph = doc.paragraphs
    st = ""
    for j in paragraph:
        st += clean(str(j.text))+" "
    data.append(st)
    temp = train_labels.loc[train_labels['File Name'] == i[:-9]]

    entities = []
    for col in data_index[1:]:
        query = str(tuple(temp[col])[0])

        if(col == 'Aggrement Start Date' or col == 'Aggrement End Date'):
            if(st.find(query) != -1):
                entities.append(
                    [st.find(query), st.find(query)+len(query), col])
                continue
            if(type(tuple(temp[col])[0]) == type(8.0) and tuple(temp[col])[0] >= 0):
                query = str(int(tuple(temp[col])[0]))
            if(st.find(query) != -1 or st.lower().find(query.lower()) != -1):
                entities.append(
                    [st.find(query), st.find(query)+len(query), col])
            print(i, col, query, type(tuple(temp[col])[0]))
            ph_1.append(i)
            ph_2.append(col)
            ph_3.append(query)
        else:
            print(i, col, query, type(tuple(temp[col])[0]))
            ph_4.append(i)
            ph_5.append(col)
            ph_6.append(query)
            count += 1
    new_data.append((st, {'entities': entities}))
print(count)

343492954-Rent-Agreement-3E.pdf.docx Aggrement Value 11500.0 <class 'float'>
343492954-Rent-Agreement-3E.pdf.docx Aggrement End Date 31.03.2015 <class 'str'>
343492954-Rent-Agreement-3E.pdf.docx Renewal Notice (Days) 30.0 <class 'float'>
343492954-Rent-Agreement-3E.pdf.docx Party One ANAND RAJ <class 'str'>
343492954-Rent-Agreement-3E.pdf.docx Party Two  BEENA PURATHAYIL MADHAVAN <class 'str'>
54945838-Rental-Agreement.pdf.docx Aggrement Value 5500.0 <class 'float'>
54945838-Rental-Agreement.pdf.docx Aggrement Start Date 21.04.2011 <class 'str'>
54945838-Rental-Agreement.pdf.docx Aggrement End Date 19.02.2012 <class 'str'>
54945838-Rental-Agreement.pdf.docx Renewal Notice (Days) 60.0 <class 'float'>
54945838-Rental-Agreement.pdf.docx Party One Asha Ramesh & Ramesh K.N <class 'str'>
54945838-Rental-Agreement.pdf.docx Party Two Sadasivuni Deepthi & Sadasivuni Kiran <class 'str'>
288024755-Rental-Agreement-1.pdf.docx Aggrement Value 5000.0 <class 'float'>
288024755-Rental-Agreement-1.pdf.

95980236-Rental-Agreement.pdf.docx Aggrement Value 9000.0 <class 'float'>
95980236-Rental-Agreement.pdf.docx Aggrement Start Date 01.04.2010 <class 'str'>
95980236-Rental-Agreement.pdf.docx Aggrement End Date 31.03.2011 <class 'str'>
95980236-Rental-Agreement.pdf.docx Renewal Notice (Days) 30.0 <class 'float'>
95980236-Rental-Agreement.pdf.docx Party One  S.Sakunthala <class 'str'>
95980236-Rental-Agreement.pdf.docx Party Two V.V.Ravi Kian <class 'str'>
294331674-Rental-Agreement.pdf.docx Aggrement Value 3500.0 <class 'float'>
294331674-Rental-Agreement.pdf.docx Aggrement Start Date 17.07.2014 <class 'str'>
294331674-Rental-Agreement.pdf.docx Aggrement End Date 17.06.2015 <class 'str'>
294331674-Rental-Agreement.pdf.docx Renewal Notice (Days) 30.0 <class 'float'>
294331674-Rental-Agreement.pdf.docx Party One MICHAEL DELA CRUZ <class 'str'>
294331674-Rental-Agreement.pdf.docx Party Two CATHERINE CABOCHA <class 'str'>
136441742-Rental-Agreement-Format.pdf.docx Aggrement Value 15000.0 <cl

In [ ]:
# pd.DataFrame(ph_1,ph_2).reset_index()
pop_df_1 = pd.DataFrame(
    {'ph1': ph_1,
     'ph2': ph_2,
     'ph3': ph_3
     })

pop_df_2 = pd.DataFrame(
    {'ph1': ph_4,
     'ph2': ph_5,
     'ph3': ph_6
     })

pop_df_1 = pop_df_1.pivot(index='ph1',columns="ph2",values="ph3").reset_index()
pop_df_2 = pop_df_2.pivot(index='ph1',columns="ph2",values="ph3").reset_index()

In [333]:
output_csv = pd.merge(pop_df_1, pop_df_2, left_on='ph1', right_on='ph1', how='outer')

In [336]:
output_csv = output_csv.rename(columns={"ph1":"FileName"})

In [344]:
os.chdir('/Users/sahmehta/Downloads/Zycus')

## Redundant lines of code to replicate the above function for validation data. This has been done to avoid function based conflicts. Can be converted into a general function later. 

In [349]:
# load validation data
data = []
data_index = list(train_labels.columns)
val_labels = pd.read_csv("ValidationSet.csv")
count = 0
new_data = []
ph_1,ph_2,ph_3,ph_4,ph_5,ph_6 = [],[],[],[],[],[]
for i in os.listdir('Validation_Data/'):
    doc = docx.Document('Validation_Data/'+i)  # Creating word reader object.
    paragraph = doc.paragraphs
    st = ""
    for j in paragraph:
        st += clean(str(j.text))+" "
    data.append(st)
    temp = val_labels.loc[val_labels['File Name'] == i[:-9]]
    # convert in proper format
    entities = []
    for col in data_index[1:]:
        query = str(tuple(temp[col])[0])
        if(col == 'Aggrement Start Date' or col == 'Aggrement End Date'):
            if(st.find(query) != -1):
                entities.append(
                    [st.find(query), st.find(query)+len(query), col])
                continue
            if(type(tuple(temp[col])[0]) == type(8.0) and tuple(temp[col])[0] >= 0):
                query = str(int(tuple(temp[col])[0]))
            if(st.find(query) != -1 or st.lower().find(query.lower()) != -1):
                entities.append(
                    [st.find(query), st.find(query)+len(query), col])
            print(i, col, query, type(tuple(temp[col])[0]))
            ph_1.append(i)
            ph_2.append(col)
            ph_3.append(query)
        else:
            print(i, col, query, type(tuple(temp[col])[0]))
            ph_4.append(i)
            ph_5.append(col)
            ph_6.append(query)
            count += 1
    new_data.append((st, {'entities': entities}))
print(count)

24158401-Rental-Agreement.pdf.docx Aggrement Value 12000 <class 'int'>
24158401-Rental-Agreement.pdf.docx Aggrement Start Date 01.04.2008 <class 'str'>
24158401-Rental-Agreement.pdf.docx Aggrement End Date 31.03.2009 <class 'str'>
24158401-Rental-Agreement.pdf.docx Renewal Notice (Days) 60.0 <class 'float'>
24158401-Rental-Agreement.pdf.docx Party One Hanumaiah  <class 'str'>
24158401-Rental-Agreement.pdf.docx Party Two  Vishal Bhardwaj  <class 'str'>
228094620-Rental-Agreement.pdf.docx Aggrement Value 15000 <class 'int'>
228094620-Rental-Agreement.pdf.docx Aggrement Start Date 07.07.2013 <class 'str'>
228094620-Rental-Agreement.pdf.docx Aggrement End Date 06.06.2014 <class 'str'>
228094620-Rental-Agreement.pdf.docx Renewal Notice (Days) 30.0 <class 'float'>
228094620-Rental-Agreement.pdf.docx Party One  KAPIL MEHROTRA  <class 'str'>
228094620-Rental-Agreement.pdf.docx Party Two .B.Kishore  <class 'str'>
195231682-This-RENTAL-AGREEMENT-is-Made-and-Executed-on-24th-Day-of-September.pdf.

In [350]:
# pd.DataFrame(ph_1,ph_2).reset_index()
pop_df_1 = pd.DataFrame(
    {'ph1': ph_1,
     'ph2': ph_2,
     'ph3': ph_3
     })

pop_df_2 = pd.DataFrame(
    {'ph1': ph_4,
     'ph2': ph_5,
     'ph3': ph_6
     })

pop_df_1 = pop_df_1.pivot(index='ph1',columns="ph2",values="ph3").reset_index()
pop_df_2 = pop_df_2.pivot(index='ph1',columns="ph2",values="ph3").reset_index()

output_csv_val = pd.merge(pop_df_1, pop_df_2, left_on='ph1', right_on='ph1', how='outer')

output_csv_val = output_csv_val.rename(columns={"ph1":"FileName"})

In [351]:
output_csv_val

ph2,FileName,Aggrement End Date,Aggrement Start Date,Aggrement Value,Party One,Party Two,Renewal Notice (Days)
0,156155545-Rental-Agreement-Kns-Home.pdf.docx,14.11.2013,15.12.2012,12000,V.K.NATARAJ,VYSHNAVI DAIRY SPECIALITIES Private Ltd,30.0
1,195231682-This-RENTAL-AGREEMENT-is-Made-and-Ex...,05.03.2014,06.04.2013,13000,C.BHAGYAMMA,JP INTERIO,30.0
2,228094620-Rental-Agreement.pdf.docx,06.06.2014,07.07.2013,15000,KAPIL MEHROTRA,.B.Kishore,30.0
3,239419594-Rental-Agreement.pdf.docx,06.06.2015,07.07.2014,9000,Abraham,Annamalai,90.0
4,24158401-Rental-Agreement.pdf.docx,31.03.2009,01.04.2008,12000,Hanumaiah,Vishal Bhardwaj,60.0
5,269135973-Udaya-Rental-Agreement.pdf.docx,31.02.2013,01.04.2014,8300,Giddappa,Pottumurthi Udayalaxmi,30.0
6,63793679-Rental-Agreement.pdf.docx,31.08.2012,NaN,9000,S Parthasarathy,Hari Kiran Tholeti,nan
7,95980236-Rental-Agreement.pdf.docx,31.03.2011,01.04.2010,9000,S.Sakunthala,V.V.Ravi Kian,30.0


## The output CSVs with required labels can be seen above and are outputted into the folder mentioned. 

In [352]:
output_csv.to_csv("OutputCSVs/Training_convert_labels.csv")

In [353]:
output_csv_val.to_csv("OutputCSVs/Validation_convert_labels.csv")